<a href="https://colab.research.google.com/github/ApoorvaChinta/assesment/blob/main/User_Engagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
users=pd.read_csv('/content/drive/MyDrive/takehome_user_engagement.csv')
users_engagement=pd.read_csv('/content/drive/MyDrive/takehome_user_engagement.csv')

In [ ]:
users.rename(columns={'object_id': 'user_id' },inplace=True)
main = pd.merge(users,users_engagement,on='user_id',how='right')

In [ ]:
main['creation_time'] = pd.to_datetime(main['creation_time'])
main['time_stamp'] = pd.to_datetime(main['time_stamp'])
main['last_session_creation_time'] = pd.to_datetime(main['last_session_creation_time'],unit='s')



In [ ]:
main.isna().sum()/len(main)*100

time_stamp_x    0.0
user_id         0.0
visited_x       0.0
time_stamp_y    0.0
visited_y       0.0
dtype: float64

In [ ]:
main.drop(columns='invited_by_user_id',axis=1,inplace=True)

In [ ]:
#Creating week
main['week'] = main['last_session_creation_time'].dt.week

In [ ]:
groupby_week = main.groupby(['user_id','week','creation_source','opted_in_to_mailing_list','enabled_for_marketing_drip','org_id'])['visited'].count().reset_index()
groupby_week.tail(10)

In [ ]:
adopted_user = groupby_week.loc[groupby_week['visited'] >= 3]
print("Total adopted users are - ",len(adopted_user))

In [ ]:
groupby_week['adopted_user'] = groupby_week['visited'].apply(lambda x: 'yes' if x >= 3 else 'No') 

In [ ]:
groupby_week.to_csv("Cleaned.csv")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
sns.countplot(y=groupby_week.adopted_user)

In [ ]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#LabelEncoding categorical columns
for column in groupby_week.columns:
   if groupby_week[column].dtype == object:
       groupby_week[column] = le.fit_transform(groupby_week[column])
#Model Creation
from sklearn.model_selection import train_test_split
X = groupby_week.drop(columns=['visited','user_id','adopted_user'],axis=1)
Y = groupby_week['adopted_user']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=124)


In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(n_estimators=150,max_depth=12,random_state=1455)
randomforest.fit(x_train,y_train)
RandomForestClassifier(max_depth=12, n_estimators=150, random_state=1455)
randomforest_score = randomforest.score(x_test,y_test)*100
print("Random Forest Score is ",randomforest_score) 



In [ ]:
#Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier(random_state=15,max_depth=0.1)
decisiontree.fit(x_train,y_train)
DecisionTreeClassifier(max_depth=0.1, random_state=15)
decisiontree_score=decisiontree.score(x_test,y_test)*100
print("Decision Tree Score is :",decisiontree_score)

In [ ]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logiregres = LogisticRegression()
logiregres.fit(x_train,y_train)
LogisticRegression()
logistic_regression_Score = logiregres.score(x_test,y_test)*100
print("Logistic Regression score is :",logistic_regression_Score)

In [ ]:
#XGBOOST Classifier

from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=2,n_estimators=240,learning_rate=0.1)
xgb.fit(x_train,y_train)

xgb_score = xgb.score(x_test,y_test)*100
print("XGBoost  Score is :",xgb_score)

Best fit model

In [ ]:
print("Random Forest Score is ",randomforest_score) 
print("Logistic Regression score is :",logistic_regression_Score)
print("Decision Tree Score is :",decisiontree_score)
print("XGBoost  Score is :",xgb_score)


In [ ]:
#Feature Importance

feature_scores = pd.Series(randomforest.feature_importances_,index=x_train.columns).sort_values(ascending=False)
feature_scores = feature_scores*100
feature_scores

In [ ]:
def find_max(nums):
  max_num=float('-inf')
  for num in nums:
    num=max_num
  return max_num